In [1]:
# imports
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df_raw = pd.read_csv('./data/GBCSG2.csv')
df_raw.head(10)

,age,estrec,horTh,menostat,pnodes,progrec,tgrade,tsize,time,event
0,70.0,66.0,no,Post,3.0,48.0,II,21.0,1814.0,True
1,56.0,77.0,yes,Post,7.0,61.0,II,12.0,2018.0,True
2,58.0,271.0,yes,Post,9.0,52.0,II,35.0,712.0,True
3,59.0,29.0,yes,Post,4.0,60.0,II,17.0,1807.0,True
4,73.0,65.0,no,Post,1.0,26.0,II,35.0,772.0,True
5,32.0,13.0,no,Pre,24.0,0.0,III,57.0,448.0,True
6,59.0,0.0,yes,Post,2.0,181.0,II,8.0,2172.0,False
7,65.0,25.0,no,Post,1.0,192.0,II,16.0,2161.0,False
8,80.0,59.0,no,Post,30.0,0.0,II,39.0,471.0,True
9,66.0,3.0,no,Post,7.0,0.0,II,18.0,2014.0,False


In [3]:
from sksurv.datasets import get_x_y

X, y = get_x_y(df_raw, attr_labels=['event','time'], pos_label=True)

# # download the dataset

# from sksurv.datasets import load_gbsg2

# X, y = load_gbsg2()

In [4]:
# number of samples, number of features/covariables
print(f'samples: {X.shape[0]}')
print(f'features/covariables: {X.shape[1]}\n')
print(f'right censored samples: {len(df_raw.query("event == False"))}')
print(f'percentage of right censored samples: {100*len(df_raw.query("event == False"))/len(df_raw):.1f}%')

samples: 686
features/covariables: 8

right censored samples: 387
percentage of right censored samples: 56.4%


In [5]:
from sklearn.model_selection import train_test_split

X_trn, X_val, y_trn, y_val = train_test_split(X, y, random_state=42)

print(f'Number of training samples: {len(y_trn)}')
print(f'Number of validation samples: {len(y_val)}')

Number of training samples: 514
Number of validation samples: 172


In [6]:
scaling_cols = [c for c in X.columns if X[c].dtype.kind in ['i', 'f']]
scaling_cols

['age', 'estrec', 'pnodes', 'progrec', 'tsize']

In [7]:
cat_cols = [c for c in X.columns if X[c].dtype.kind not in ["i", "f"]]
cat_cols

['horTh', 'menostat', 'tgrade']

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

preprocessor = ColumnTransformer(
    [('cat-preprocessor', OrdinalEncoder(), cat_cols),
    ('standard-scaler', StandardScaler(), scaling_cols)],
    remainder='passthrough', sparse_threshold=0)

In [9]:
from sklearn.pipeline import make_pipeline
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

cox = make_pipeline(preprocessor, CoxPHSurvivalAnalysis())
cox.fit(X_trn, y_trn)

ci_cox = concordance_index_censored(y_val["event"], y_val["time"], cox.predict(X_val))
print(f'The c-index of Cox is given by {ci_cox[0]:.3f}')

The c-index of Cox is given by 0.635


In [10]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'coxphsurvivalanalysis__alpha': reciprocal(0.1, 100),
}

model_random_search = RandomizedSearchCV(cox, param_distributions=param_distributions, n_iter=50, n_jobs=-1, cv=5, random_state=42)
model_random_search.fit(X_trn, y_trn)

print(
    f"The c-index of Cox using a {model_random_search.__class__.__name__} is "
    f"{model_random_search.score(X_val, y_val):.3f}")
print(
    f"The best set of parameters is: {model_random_search.best_params_}"
)

The c-index of Cox using a RandomizedSearchCV is 0.653
The best set of parameters is: {'coxphsurvivalanalysis__alpha': 53.45166110646816}


In [11]:
from sklearn.pipeline import make_pipeline
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

gbc = make_pipeline(preprocessor, GradientBoostingSurvivalAnalysis(random_state=42))

gbc.fit(X_trn, y_trn)

ci_gbc = concordance_index_censored(y_val["event"], y_val["time"], gbc.predict(X_val))
print(f'The c-index of Gradient Boosting Cox PH is given by {ci_gbc[0]:.3f}')

The c-index of Gradient Boosting Cox PH is given by 0.641


In [12]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

class reciprocal_int:
    def __init__(self, a, b):
        self._distribution = reciprocal(a, b)

    def rvs(self, *args, **kwargs):
        return self._distribution.rvs(*args, **kwargs).astype(int)

param_distributions = {
    'gradientboostingsurvivalanalysis__learning_rate': reciprocal(0.001, 1),
    'gradientboostingsurvivalanalysis__max_depth': reciprocal_int(5, 50),
    'gradientboostingsurvivalanalysis__min_samples_leaf': reciprocal_int(1, 40),
}

model_random_search = RandomizedSearchCV(gbc, param_distributions=param_distributions, n_iter=50, n_jobs=-1, cv=5, random_state=42)
model_random_search.fit(X_trn, y_trn)

print(
    f"The c-index of gradient boosting Cox PH using a {model_random_search.__class__.__name__} is "
    f"{model_random_search.score(X_val, y_val):.3f}")
print(
    f"The best set of parameters is: {model_random_search.best_params_}"
)

The c-index of gradient boosting Cox PH using a RandomizedSearchCV is 0.664
The best set of parameters is: {'gradientboostingsurvivalanalysis__learning_rate': 0.04366473592979633, 'gradientboostingsurvivalanalysis__max_depth': 7, 'gradientboostingsurvivalanalysis__min_samples_leaf': 35}


In [13]:
X_trn = preprocessor.fit_transform(X_trn)
X_val = preprocessor.transform(X_val)

In [14]:
from pysurvival.models.survival_forest import RandomSurvivalForestModel

rsf = RandomSurvivalForestModel(num_trees=20)
rsf.fit(X_trn, y_trn["time"], y_trn["event"])
ci_rsf = concordance_index_censored(y_val["event"], y_val["time"], rsf.predict_risk(X_val))

print(f'The c-index of Random survival forest is given by {ci_rsf[0]:.3f}')

The c-index of Random survival forest is given by 0.672


In [15]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel

structure = [ {'activation': 'ReLU', 'num_units': 60}, {'activation': 'ReLU', 'num_units': 10}, ]

# Building the model
nonlinear_coxph = NonLinearCoxPHModel(structure=structure)
nonlinear_coxph.fit(X_trn, y_trn["time"], y_trn["event"])

ci_deepsurv = concordance_index_censored(y_val["event"], y_val["time"], nonlinear_coxph.predict_risk(X_val))

print(f'The c-index of DeepSurv is given by {ci_deepsurv[0]:.3f}')

The c-index of DeepSurv is given by 0.684
